## Training Data

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rama\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Rama\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Rama\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']
intents_file = open('intents.json').read()
intents = json.loads(intents_file)

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        #add documents in the corpus
        documents.append((word, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents)

[(['Halo'], 'greeting'), (['Apa', 'kabar'], 'greeting'), (['Ada', 'orang', 'disana', '?'], 'greeting'), (['Hei'], 'greeting'), (['Woy'], 'greeting'), (['Kuy'], 'greeting'), (['Salam'], 'greeting'), (['Assalamualaikum'], 'salam'), (['Bye'], 'goodbye'), (['Sampai', 'Jumpa'], 'goodbye'), (['Selamat', 'Tinggal'], 'goodbye'), (['Dadah'], 'goodbye'), (['Dahh'], 'goodbye'), (['Terima', 'kasih'], 'thanks'), (['Makasih'], 'thanks'), (['Thanks'], 'thanks'), (['Terima', 'Kasih', 'Banyak'], 'thanks'), (['Terima', 'kasih', 'telah', 'membantu', 'saya'], 'thanks'), (['Bisakah', 'kamu', 'membantuku', '?'], 'options'), (['Apa', 'yang', 'bisa', 'kamu', 'lakukan', '?'], 'options'), (['Apa', 'yang', 'kamu', 'tawarkan', '?'], 'options'), (['Bagaimana', 'kamu', 'sangat', 'membantu', '?'], 'options'), (['Kamu', 'bisa', 'bantu', 'aku', '?'], 'options'), (['Bagaimana', 'cara', 'memeriksa', 'reaksi', 'obat', 'yang', 'merugikan', '?'], 'adverse_drug'), (['Buka', 'modul', 'obat', 'yang', 'merugikan'], 'adverse_dr

In [4]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

48 documents
10 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'salam', 'thanks']
85 unique lemmatized words ['ada', 'aku', 'apa', 'apotek', 'assalamualaikum', 'bagaimana', 'bantu', 'banyak', 'berhubungan', 'beri', 'bisa', 'bisakah', 'buka', 'buruk', 'bye', 'cara', 'cari', 'carikan', 'cocok', 'dadah', 'daftar', 'dahh', 'darah', 'data', 'dengan', 'detail', 'disana', 'efek', 'entri', 'halo', 'hasil', 'hei', 'id', 'ingin', 'jumpa', 'kabar', 'kamu', 'kasih', 'kuy', 'lakukan', 'makasih', 'mana', 'manajemen', 'membantu', 'membantuku', 'memeriksa', 'memiliki', 'memindahkan', 'mencari', 'mencatat', 'menyebabkan', 'merugikan', 'modul', 'muat', 'obat', 'obat-obatan', 'orang', 'pasien', 'pencarian', 'perilaku', 'reaksi', 'riwayat', 'rumah', 'sakit', 'salam', 'sampai', 'samping', 'sangat', 'saya', 'selamat', 'semua', 'tampilkan', 'tawarkan', 'tekanan', 'telah', 'temukan', 'terdekat', 'terima', 'thanks', 't

In [5]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [6]:
# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for word in words:
        bag.append(1) if word in pattern_words else bag.append(0)
        
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [7]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\Rama\AppData\Local\Temp\ipykernel_9208\810444173.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [8]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [9]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

C:\Users\Rama\anaconda3\envs\firstenv\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [10]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
10/10 [==============================] - 1s 5ms/step - loss: 2.3341 - accuracy: 0.1042
Epoch 2/200
10/10 [==============================] - 0s 5ms/step - loss: 2.2936 - accuracy: 0.1458
Epoch 3/200
10/10 [==============================] - 0s 5ms/step - loss: 2.1647 - accuracy: 0.1875
Epoch 4/200
10/10 [==============================] - 0s 4ms/step - loss: 2.0290 - accuracy: 0.3333
Epoch 5/200
10/10 [==============================] - 0s 4ms/step - loss: 1.9081 - accuracy: 0.4167
Epoch 6/200
10/10 [==============================] - 0s 4ms/step - loss: 1.9057 - accuracy: 0.4375
Epoch 7/200
10/10 [==============================] - 0s 4ms/step - loss: 1.7601 - accuracy: 0.5000
Epoch 8/200
10/10 [==============================] - 0s 5ms/step - loss: 1.6204 - accuracy: 0.5417
Epoch 9/200
10/10 [==============================] - 0s 5ms/step - loss: 1.5486 - accuracy: 0.5833
Epoch 10/200
10/10 [==============================] - 0s 5ms/step - loss: 1.3740 - accuracy: 0.6250
Epoch 11/

10/10 [==============================] - 0s 4ms/step - loss: 0.0903 - accuracy: 0.9583
Epoch 84/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0440 - accuracy: 1.0000
Epoch 85/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0468 - accuracy: 0.9792
Epoch 86/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0794 - accuracy: 0.9792
Epoch 87/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0684 - accuracy: 1.0000
Epoch 88/200
10/10 [==============================] - 0s 4ms/step - loss: 0.1118 - accuracy: 0.9792
Epoch 89/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0593 - accuracy: 1.0000
Epoch 90/200
10/10 [==============================] - 0s 4ms/step - loss: 0.2131 - accuracy: 0.9167
Epoch 91/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0455 - accuracy: 1.0000
Epoch 92/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0818 - accuracy: 1.0000
Epoch 93/200


10/10 [==============================] - 0s 4ms/step - loss: 0.0295 - accuracy: 1.0000
Epoch 165/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0479 - accuracy: 0.9583
Epoch 166/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0372 - accuracy: 0.9792
Epoch 167/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0335 - accuracy: 1.0000
Epoch 168/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0061 - accuracy: 1.0000
Epoch 169/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0127 - accuracy: 1.0000
Epoch 170/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0219 - accuracy: 1.0000
Epoch 171/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0137 - accuracy: 1.0000
Epoch 172/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0523 - accuracy: 0.9792
Epoch 173/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0184 - accuracy: 1.0000
Epoc

## GUI Interface

In [11]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import tkinter
from tkinter import *
import json

In [12]:
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [13]:
def clean_up_sentence(sentence):
    # tokenize the pattern - splitting words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stemming every word - reducing to base form
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [14]:
# return bag of words array: 0 or 1 for words that exist in sentence
def bag_of_words(sentence, words, show_details=True):
    # tokenizing patterns
    sentence_words = clean_up_sentence(sentence)
    # bag of words - vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,word in enumerate(words):
            if word == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % word)
    return(np.array(bag))

In [15]:
def predict_class(sentence):
    # filter below  threshold predictions
    p = bag_of_words(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sorting strength probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [16]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [17]:
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatBox.config(state=NORMAL)
        ChatBox.insert(END, "You: " + msg + '\n\n')
        ChatBox.config(foreground="#446665", font=("Verdana", 12 ))
    
        ints = predict_class(msg)
        res = getResponse(ints, intents)
        
        ChatBox.insert(END, "Bot: " + res + '\n\n')
            
        ChatBox.config(state=DISABLED)
        ChatBox.yview(END)

In [18]:
root = Tk()
root.title("Chatbot")
root.geometry("400x500")
root.resizable(width=FALSE, height=FALSE)

''

In [19]:
#Create Chat window
ChatBox = Text(root, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatBox.config(state=DISABLED)

In [20]:
#Bind scrollbar to Chat window
scrollbar = Scrollbar(root, command=ChatBox.yview, cursor="heart")
ChatBox['yscrollcommand'] = scrollbar.set

In [21]:
#Create Button to send message
SendButton = Button(root, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#f9a602", activebackground="#3c9d9b",fg='#000000',
                    command= send )

In [22]:
#Create the box to enter message
EntryBox = Text(root, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)

In [23]:
#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatBox.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

root.mainloop()

1/1 [==============================] - 0s 26ms/step
